In [7]:
import numpy as np
from keras import backend as K
from keras.engine import Input, Model
from keras.layers import Conv3D, MaxPooling3D, UpSampling3D, Activation, BatchNormalization, PReLU, Deconvolution3D
K.set_image_data_format("channels_first")

try:
    from keras.engine import merge
except ImportError:
    from keras.layers.merge import concatenate


def unet_model_3d(input_shape, pool_size=(2, 2, 2), n_labels=1, deconvolution=False,
                  depth=4, n_base_filters=32, batch_normalization=False, activation_name="sigmoid"):
    """
    Builds the 3D UNet Keras model.f
    :param metrics: List metrics to be calculated during model training (default is dice coefficient).
    :param include_label_wise_dice_coefficients: If True and n_labels is greater than 1, model will report the dice
    coefficient for each label as metric.
    :param n_base_filters: The number of filters that the first layer in the convolution network will have. Following
    layers will contain a multiple of this number. Lowering this number will likely reduce the amount of memory required
    to train the model.
    :param depth: indicates the depth of the U-shape for the model. The greater the depth, the more max pooling
    layers will be added to the model. Lowering the depth may reduce the amount of memory required for training.
    :param input_shape: Shape of the input data (n_chanels, x_size, y_size, z_size). The x, y, and z sizes must be
    divisible by the pool size to the power of the depth of the UNet, that is pool_size^depth.
    :param pool_size: Pool size for the max pooling operations.
    :param n_labels: Number of binary labels that the model is learning.
    :param initial_learning_rate: Initial learning rate for the model. This will be decayed during training.
    :param deconvolution: If set to True, will use transpose convolution(deconvolution) instead of up-sampling. This
    increases the amount memory required during training.
    :return: Untrained 3D UNet Model
    """
    inputs = Input(input_shape)
    current_layer = inputs
    levels = list()
    num_layer = 0

    # add levels with max pooling
    for layer_depth in range(depth):
        layer1 = create_convolution_block(input_layer=current_layer, n_filters=n_base_filters*(2**layer_depth),
                                          batch_normalization=batch_normalization, layer_depth=num_layer)
        num_layer += 1
        layer2 = create_convolution_block(input_layer=layer1, n_filters=n_base_filters*(2**layer_depth)*2,
                                          batch_normalization=batch_normalization, layer_depth=num_layer)
        num_layer += 1
        if layer_depth < depth - 1:
            current_layer = MaxPooling3D(pool_size=pool_size)(layer2)
            levels.append([layer1, layer2, current_layer])
        else:
            current_layer = layer2
            levels.append([layer1, layer2])

    # add levels with up-convolution or up-sampling
    for layer_depth in range(depth-2, -1, -1):
        up_convolution = get_up_convolution(pool_size=pool_size, deconvolution=deconvolution,
                                            n_filters=current_layer._keras_shape[1])(current_layer)
        concat = concatenate([up_convolution, levels[layer_depth][1]], axis=1)
        current_layer = create_convolution_block(n_filters=levels[layer_depth][1]._keras_shape[1], layer_depth=num_layer,
                                                 input_layer=concat, batch_normalization=batch_normalization)
        num_layer += 1
        current_layer = create_convolution_block(n_filters=levels[layer_depth][1]._keras_shape[1], layer_depth=num_layer,
                                                 input_layer=current_layer,
                                                 batch_normalization=batch_normalization)
        num_layer += 1

    final_convolution = Conv3D(n_labels, (1, 1, 1))(current_layer)
    act = Activation(activation_name)(final_convolution)
    model = Model(inputs=inputs, outputs=act)

    return model

def create_convolution_block(input_layer, n_filters, batch_normalization=False, kernel=(3, 3, 3), activation=None,
                             padding='same', strides=(1, 1, 1), instance_normalization=False, layer_depth=None):
    """
    :param strides:
    :param input_layer:
    :param n_filters:
    :param batch_normalization:
    :param kernel:
    :param activation: Keras activation layer to use. (default is 'relu')
    :param padding:
    :return:
    """
    layer = Conv3D(n_filters, kernel, padding=padding, strides=strides, name="depth_"+str(layer_depth)+"_conv")(input_layer)
    if batch_normalization:
        layer = BatchNormalization(axis=1, name="depth_"+str(layer_depth)+"_bn")(layer)
    elif instance_normalization:
        try:
            from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
        except ImportError:
            raise ImportError("Install keras_contrib in order to use instance normalization."
                              "\nTry: pip install git+https://www.github.com/farizrahman4u/keras-contrib.git")
        layer = InstanceNormalization(axis=1, name="depth_"+str(layer_depth)+"_in")(layer)
    if activation is None:
        return Activation('relu', name="depth_"+str(layer_depth)+"_relu")(layer)
    else:
        return activation()(layer)


def compute_level_output_shape(n_filters, depth, pool_size, image_shape):
    """
    Each level has a particular output shape based on the number of filters used in that level and the depth or number 
    of max pooling operations that have been done on the data at that point.
    :param image_shape: shape of the 3d image.
    :param pool_size: the pool_size parameter used in the max pooling operation.
    :param n_filters: Number of filters used by the last node in a given level.
    :param depth: The number of levels down in the U-shaped model a given node is.
    :return: 5D vector of the shape of the output node 
    """
    output_image_shape = np.asarray(np.divide(image_shape, np.power(pool_size, depth)), dtype=np.int32).tolist()
    return tuple([None, n_filters] + output_image_shape)


def get_up_convolution(n_filters, pool_size, kernel_size=(2, 2, 2), strides=(2, 2, 2),
                       deconvolution=False):
    if deconvolution:
        return Deconvolution3D(filters=n_filters, kernel_size=kernel_size,
                               strides=strides)
    else:
        return UpSampling3D(size=pool_size)

In [8]:

import matplotlib.pyplot as plt
from sklearn.metrics import  f1_score, precision_score, recall_score, accuracy_score, roc_curve, auc, roc_auc_score, confusion_matrix as CM

import os
import logging
import numpy as np
import tensorflow as tf

import keras
from keras.models import * 
from keras.optimizers import Adam 
from tensorflow.keras.callbacks import ReduceLROnPlateau
from keras.layers import Dense, GlobalAveragePooling3D, Dropout, BatchNormalization

In [9]:
# cretae the binary input 
def data_filter(data, label, exclu):
    idx = np.where(label!= exclu)[0]
    #print(len(idx))
    data_new = data[idx]
    label_new = label[idx]
    #print(data_new.shape)
    print(np.unique(label_new, return_counts=True))
    return data_new, label_new

# onehot encode labels for binary classifications
def onehot_bi(y):
    from sklearn.preprocessing import OneHotEncoder
    onehot_encoder = OneHotEncoder(sparse=False)
    y = y.reshape(len(y), 1)
    y_encoded = onehot_encoder.fit_transform(y)
    return y_encoded

# onehot encode labels for 3-way classifications
def onehot_tri(y):
    from keras.utils import to_categorical
    return to_categorical(y)

# view the distribution of class labels of the input data
def showpercentage(array):
    pcn = array[1][0]/np.sum(array[1])
    pmci = array[1][1]/np.sum(array[1])
    pad = array[1][2]/np.sum(array[1])
    print(str(pad) + " percent of the data has AD label")
    print(str(pcn) + " percent of the data has CN label")
    print(str(pmci) + " percent of the data has MCI label")
# visualizatio of model traning and model performance 

# visualize the training and validation performance
def plot_history(data_list, label_list, title, ylabel, name):

    epochs = range(1, len(data_list[0]) + 1)

    for data, label in zip(data_list, label_list):
        plt.plot(epochs, data, label=label)
    plt.title(title, pad = 10, fontsize='large')
    plt.xlabel('Epochs', labelpad=10)
    plt.ylabel(ylabel)
    plt.legend()
    plt.savefig(name, dpi=300, bbox_inches='tight')
    plt.show()
#%%

# model evaluation
    
# evaluate model performance - binary classifications
def evaluate_binary(X_test, y_test, model, name):
#     with tf.Session() as sess:
#         sess.run(tf.global_variables_initializer())
    test_y_prob = model.predict(X_test)
    print(test_y_prob)
    test_y_pred = np.argmax(test_y_prob, axis = 1)
    test_y_true = np.argmax(y_test, axis = 1) 
    # accuracy
#     with tf.Session() as sess:
#         sess.run(tf.global_variables_initializer())
    loss, acc = model.evaluate(X_test, y_test)
    # AUC
    pos_prob = test_y_prob[:,1]
    auc_score = roc_auc_score(test_y_true, pos_prob)
    # precision, recall, specificity, and f1_score
    p = precision_score(test_y_true, test_y_pred)
    r = recall_score(test_y_true, test_y_pred)
    f1 = f1_score(test_y_true, test_y_pred)
#     sen, spe, _ = sss(test_y_true, test_y_pred, average="binary")
    print(test_y_true, test_y_pred)
    # print results
    print("Test accuracy:", acc)
    print("Test AUC is: ", auc_score)
    print("Test confusion matrix: \n", CM(test_y_true, test_y_pred))
    print("Precision: ", p)
    print("Recall: ", r)
#     print("Specificity: ", spe)
    print("f1_score: ", f1)

    # plot and save roc curve
    pos_prob = test_y_prob[:,1]
    fpr, tpr, thresholds = roc_curve(test_y_true, pos_prob)
    ns_probs = [0 for _ in range(len(test_y_prob))]
    ns_fpr, ns_tpr, _ = roc_curve(test_y_true, ns_probs)
    plt.axis([0,1,0,1]) 
    plt.plot(fpr,tpr, marker = '.', color = 'darkorange', label = 'Model AUC (area = {:.2f})'.format(auc_score)) 
    plt.plot(ns_fpr, ns_tpr, color = 'royalblue', linestyle='--')
    plt.legend()
    plt.xlabel('False Positive Rate') 
    plt.ylabel('True Positive Rate') 
    plt.savefig(name, dpi=300, bbox_inches='tight')
    plt.show()
    
# evaluate model performance - 3 way classifiations
def evaluate_3way(X_test, y_test, model):
    test_y_prob = model.predict(X_test)
    test_y_pred = np.argmax(test_y_prob, axis = 1)
    test_y_true = np.argmax(y_test, axis = 1) 
    print(test_y_prob)
    # accuracy
    loss, acc = model.evaluate(X_test, y_test)
    # precision, recall, specificity, and f1_score
    p = precision_score(test_y_true, test_y_pred, average="macro")
    r = recall_score(test_y_true, test_y_pred, average="macro")
    f1 = f1_score(test_y_true, test_y_pred, average="macro")
#     sen,spe,_ = sss(test_y_true, test_y_pred, average="macro")
    print(test_y_true, test_y_pred)
    print("Test accuracy:", acc)
    print("Test confusion matrix: \n", CM(test_y_true, test_y_pred))
    print("Precision: ", p)
    print("Recall: ", r)
#     print("Specificity: ", spe)
    print("f1_score: ", f1)

In [10]:
# build the Genesis model(set %tensorflow_version 1.x for training Genesis)
def run_genesis(X_train, y_train, X_valid = None, y_valid = None, 
             final = False, out = 2,
             dr = 0.02, n_epochs = 150, batch_size = 15):
  
    input_channels, input_rows, input_cols, input_deps = 1, 64, 64, 64

    weight_dir = 'Genesis_Chest_CT.h5'

    models_genesis = unet_model_3d((input_channels, input_rows, input_cols, input_deps), batch_normalization=True)
    plot_model(models_genesis, to_file='test2.png', show_shapes=True)
    print("Load pre-trained Models Genesis weights from {}".format(weight_dir))
    models_genesis.load_weights(weight_dir)

    x = models_genesis.get_layer('depth_7_relu').output
    x = GlobalAveragePooling3D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(dr)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(dr)(x)

    output = Dense(out, activation = 'softmax')(x)

    # model optimization
    # opt = Adam(learning_rate = lr)
    model = keras.models.Model(inputs=models_genesis.input, outputs=output)
    model.compile(optimizer="adam", loss = 'categorical_crossentropy', metrics=['accuracy'])
    cb = ReduceLROnPlateau(monitor = 'val_loss', 
                         factor = 0.5, patience = 5, 
                         verbose = 1, mode = 'min')


    # model training and fine-tuning
    if not final:
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
#             plot_model(model, to_file='test.png', show_shapes=True)
            hist = model.fit(X_train, y_train,
                             batch_size = batch_size, 
                             epochs = n_epochs,
                             callbacks=[cb],
                             validation_data = (X_valid, y_valid), 
                             shuffle = True)
            evaluate_3way(X_valid, y_valid, model)
            return model, hist

    # model final training for testing (train + valid combined)
    else:
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            hist = model.fit(X_train, y_train,
                      batch_size = batch_size, 
                      epochs = n_epochs,
                      callbacks=[cb],
                      shuffle = True)


            return model, hist

In [11]:
"""  Load in the input - original data """

Xtr = np.load("preprocess/input/random_split/train_data.npy", allow_pickle = True)
ytr = np.load("preprocess/input/random_split/train_label.npy", allow_pickle = True)
print(Xtr.shape)
print(ytr.shape)

Xts = np.load("preprocess/input/random_split/test_data.npy", allow_pickle = True)
yts = np.load("preprocess/input/random_split/test_label.npy", allow_pickle = True)
print(Xts.shape)
print(yts.shape)

Xval = np.load("preprocess/input/random_split/val_data.npy", allow_pickle = True)
yval = np.load("preprocess/input/random_split/val_label.npy", allow_pickle = True)
print(Xval.shape)
print(yval.shape)

print("Train:")
showpercentage(np.unique(ytr, return_counts=True))
print()
print("Validation:")
showpercentage(np.unique(yval, return_counts=True))
print()
print("Test")
showpercentage(np.unique(yts, return_counts=True))

(356, 64, 64, 64)
(356,)
(86, 64, 64, 64)
(86,)
(90, 64, 64, 64)
(90,)
Train:
0.2893258426966292 percent of the data has AD label
0.41853932584269665 percent of the data has CN label
0.29213483146067415 percent of the data has MCI label

Validation:
0.28888888888888886 percent of the data has AD label
0.4222222222222222 percent of the data has CN label
0.28888888888888886 percent of the data has MCI label

Test
0.29069767441860467 percent of the data has AD label
0.4186046511627907 percent of the data has CN label
0.29069767441860467 percent of the data has MCI label


In [ ]:
"""  NC vs. AD - transfer learning """

# create input for binary classification of NC vs. AD
Xtr_ncad, ytr_ncad = data_filter(Xtr, ytr, 1)
Xval_ncad, yval_ncad = data_filter(Xval, yval, 1)
Xts_ncad, yts_ncad = data_filter(Xts, yts, 1)

# reshape the input
X_train = np.transpose(Xtr_ncad.reshape(-1,64,64,64,1), (0, 4, 1, 2, 3)) 
X_test = np.transpose(Xts_ncad.reshape(-1,64,64,64,1), (0, 4, 1, 2, 3))
X_val = np.transpose(Xval_ncad.reshape(-1,64,64,64,1), (0, 4, 1, 2, 3))

# one hot encode the target labels 
y_train = onehot_bi(ytr_ncad)
y_test = onehot_bi(yts_ncad)
y_val = onehot_bi(yval_ncad)


# model training
model, hist = run_genesis(X_train, y_train, X_val, y_val)

(array([0, 2]), array([149, 103]))
(array([0, 2]), array([38, 26]))
(array([0, 2]), array([36, 25]))
Load pre-trained Models Genesis weights from Genesis_Chest_CT.h5
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 252 samples, validate on 64 samples
Epoch 1/150


In [5]:
import numpy as np
import matplotlib.pyplot as plt
# from imblearn.metrics import sensitivity_specificity_support as sss
from sklearn.metrics import  f1_score, precision_score, recall_score, accuracy_score, roc_curve, auc, roc_auc_score, confusion_matrix as CM

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Input, Dense, Conv3D, GlobalAveragePooling3D,  MaxPooling3D, LeakyReLU, BatchNormalization, Dropout, Flatten, Activation, Reshape,  Conv3DTranspose, UpSampling3D
from tensorflow.keras.regularizers import l2, l1, l1_l2

In [6]:

# cretae the binary input 
def data_filter(data, label, exclu):
    idx = np.where(label!= exclu)[0]
    #print(len(idx))
    data_new = data[idx]
    label_new = label[idx]
    #print(data_new.shape)
    print(np.unique(label_new, return_counts=True))
    return data_new, label_new

# onehot encode labels for binary classifications
def onehot_bi(y):
    from sklearn.preprocessing import OneHotEncoder
    onehot_encoder = OneHotEncoder(sparse=False)
    y = y.reshape(len(y), 1)
    y_encoded = onehot_encoder.fit_transform(y)
    return y_encoded

# onehot encode labels for 3-way classifications
def onehot_tri(y):
    from keras.utils import to_categorical
    return to_categorical(y)

# view the distribution of class labels of the input data
def showpercentage(array):
    pcn = array[1][0]/np.sum(array[1])
    pmci = array[1][1]/np.sum(array[1])
    pad = array[1][2]/np.sum(array[1])
    print(str(pad) + " percent of the data has AD label")
    print(str(pcn) + " percent of the data has CN label")
    print(str(pmci) + " percent of the data has MCI label")


#%%
# models

# build the baseline model
def run_base(X_train, y_train, X_valid = None, y_valid = None, 
             final = False, out = 2,
             dr = 0.02, lr = 0.00001, 
             breg = l2(0.0001), areg = None, 
             n_epochs = 30, batch_size = 15):
  
    dim = (64, 64, 64, 1)

    model = Sequential()
    model.add(Conv3D(32, kernel_size=(5,5,5),  kernel_initializer='he_uniform', bias_regularizer=breg, input_shape=dim))
    #model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))

    model.add(Conv3D(64, kernel_size=(5,5,5),  bias_regularizer=breg, kernel_initializer='he_uniform'))
    #model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(2,2,2)))


    model.add(Conv3D(128, kernel_size=(5,5,5),  bias_regularizer=breg, kernel_initializer='he_uniform'))
    #model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(2,2,2)))

    model.add(Dropout(dr))

    model.add(Flatten())
    model.add(Dense(512, bias_regularizer=breg,   kernel_initializer='he_uniform'))
    #model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Dropout(dr))

    model.add(Dense(256, bias_regularizer=breg,   kernel_initializer='he_uniform'))
    #model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Dense(out, activation='softmax', activity_regularizer=areg))

    # model optimization
    opt = Adam(lr = lr)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy']) 

    cb = ReduceLROnPlateau(monitor = 'val_loss', 
                         factor = 0.5, patience = 5, 
                         verbose = 1, epsilon = 1e-4, mode = 'min')
    
  # model training and fine-tuning
    if not final:
        hist = model.fit(X_train, y_train,
                     batch_size = batch_size, 
                     epochs = n_epochs,
                     callbacks=[cb],
                     validation_data = (X_valid, y_valid), 
                     shuffle = True)
  
  # model final training for testing (train + valid combined)
    else:
        hist = model.fit(X_train, y_train,
                  batch_size = batch_size, 
                  epochs = n_epochs,
                  callbacks=[cb],
                  shuffle = True)


    return model, hist


# build the convolutional autoencoder
def run_cae(X_train,  X_val, lr = 0.0001, 
        n_epochs = 100, batch_size = 10):
  
    dim = (64,64,64,1)

    inp = Input(dim)
    # Encoder
    e = Conv3D(32, (3, 3, 3), strides = 2, activation='elu',  kernel_initializer='he_uniform', padding = "same")(inp)
    e = Conv3D(64, (3, 3, 3), strides = 2, activation='elu',  kernel_initializer='he_uniform', padding = "same")(e)
    e = Conv3D(1, (3, 3, 3), strides = 2,activation='elu',  kernel_initializer='he_uniform', padding = "same", name = "bottleneck")(e)

    #DECODER
    d = Conv3DTranspose(64,(3,3,3), strides = 2, kernel_initializer='he_uniform', activation='elu', padding = "same")(e)
    d = BatchNormalization()(d)
    d = Conv3DTranspose(16,(3,3,3), strides=2,  kernel_initializer='he_uniform', activation='elu', padding = "same")(d)
    d = BatchNormalization()(d)
    d = Conv3DTranspose(16,(3,3,3), strides=2,  kernel_initializer='he_uniform', activation='elu', padding = "same")(d)
    decoded = Conv3D(1, (3, 3, 3), activation='sigmoid', padding='same')(d)

    ae = Model(inp, decoded)
    
    # model optimization
    opt_ae = Adam(lr = lr)
    ae.compile(optimizer = opt_ae, loss = "mse")
    cb = ReduceLROnPlateau(monitor = 'val_loss', 
                         factor = 0.9, patience = 3, 
                         verbose = 1, mode = 'min')
    plot_model(ae, to_file='test3.png', show_shapes=True)
    #Train it by providing training images
    ae.fit(X_train, X_train, 
         batch_size = batch_size, 
         epochs = n_epochs, 
         validation_data = (X_val, X_val), 
         verbose = 1,
         callbacks = [cb])

    return ae

# build the adapted baseline model for training with dimension reduced data
def run_adpbase(X_train, y_train, X_valid = None, y_valid = None, 
             final = False, out = 2,
             dr = 0.2, lr = 0.0001, 
             breg = l2(0.00001), wreg = l2(0.00001), areg = l1(0.00001), 
             n_epochs = 100, batch_size = 25):
  
    dim = (8,8,8,1)

    model = Sequential()
    model.add(Conv3D(32, kernel_size=(2,2,2), kernel_initializer='he_uniform',bias_regularizer=breg, kernel_regularizer=wreg,input_shape=dim))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Conv3D(64, kernel_size=(2,2,2), kernel_initializer='he_uniform',bias_regularizer=breg, kernel_regularizer=wreg,input_shape=dim))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(2,2,2)))
    model.add(Dropout(dr))

    model.add(Conv3D(128, kernel_size=(2,2,2), kernel_initializer='he_uniform',bias_regularizer=breg, kernel_regularizer=wreg,input_shape=dim))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(2,2,2)))
    model.add(Dropout(dr))

    model.add(Flatten())
    model.add(Dense(512,kernel_initializer='he_uniform', kernel_regularizer=wreg,bias_regularizer=breg))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(dr))

    model.add(Dense(out, activation='softmax', activity_regularizer=areg))

    # model optimization
    opt = Adam(lr=lr)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy']) 
    cb = ReduceLROnPlateau(monitor = 'val_loss', 
                         factor = 0.5, patience = 5, 
                         verbose = 1, epsilon = 1e-4, mode = 'min')
    plot_model(model, to_file='test4.png', show_shapes=True)
    # model training and fine-tuning
    if not final:
        hist = model.fit(X_train, y_train,
                         batch_size = batch_size, 
                         epochs = n_epochs,
                         callbacks=[cb],
                         validation_data = (X_valid, y_valid), 
                         shuffle = True)

    # model final training for testing (train + valid combined)
    else:
        hist = model.fit(X_train, y_train,
                  batch_size = batch_size, 
                  epochs = n_epochs,
                  callbacks = [cb],
                  shuffle = True)


    return model, hist

#%%
  
# visualizatio of model traning and model performance 

# visualize the training and validation performance
def plot_history(data_list, label_list, title, ylabel, name):

    epochs = range(1, len(data_list[0]) + 1)

    for data, label in zip(data_list, label_list):
        plt.plot(epochs, data, label=label)
    plt.title(title, pad = 10, fontsize='large')
    plt.xlabel('Epochs', labelpad=10)
    plt.ylabel(ylabel)
    plt.legend()
    plt.savefig(name, dpi=300, bbox_inches='tight')
    plt.show()
#%%

# model evaluation
    
# evaluate model performance - binary classifications
def evaluate_binary(X_test, y_test, model, name):
    test_y_prob = model.predict(X_test)
    print(test_y_prob)
    test_y_pred = np.argmax(test_y_prob, axis = 1)
    test_y_true = np.argmax(y_test, axis = 1) 
    # accuracy
    loss, acc = model.evaluate(X_test, y_test)
    # AUC
    pos_prob = test_y_prob[:,1]
    auc_score = roc_auc_score(test_y_true, pos_prob)
    # precision, recall, specificity, and f1_score
    p = precision_score(test_y_true, test_y_pred)
    r = recall_score(test_y_true, test_y_pred)
    f1 = f1_score(test_y_true, test_y_pred)
#     sen, spe, _ = sss(test_y_true, test_y_pred, average="binary")
    print(test_y_true, test_y_pred)
    # print results
    print("Test accuracy:", acc)
    print("Test AUC is: ", auc_score)
    print("Test confusion matrix: \n", CM(test_y_true, test_y_pred))
    print("Precision: ", p)
    print("Recall: ", r)
#     print("Specificity: ", spe)
    print("f1_score: ", f1)

    # plot and save roc curve
    pos_prob = test_y_prob[:,1]
    fpr, tpr, thresholds = roc_curve(test_y_true, pos_prob)
    ns_probs = [0 for _ in range(len(test_y_prob))]
    ns_fpr, ns_tpr, _ = roc_curve(test_y_true, ns_probs)
    plt.axis([0,1,0,1]) 
    plt.plot(fpr,tpr, marker = '.', color = 'darkorange', label = 'Model AUC (area = {:.2f})'.format(auc_score)) 
    plt.plot(ns_fpr, ns_tpr, color = 'royalblue', linestyle='--')
    plt.legend()
    plt.xlabel('False Positive Rate') 
    plt.ylabel('True Positive Rate') 
    plt.savefig(name, dpi=300, bbox_inches='tight')
    plt.show()
    
# evaluate model performance - 3 way classifiations
def evaluate_3way(X_test, y_test, model):
    test_y_prob = model.predict(X_test)
    test_y_pred = np.argmax(test_y_prob, axis = 1)
    test_y_true = np.argmax(y_test, axis = 1) 
    print(test_y_prob)
    # accuracy
    loss, acc = model.evaluate(X_test, y_test)
    # precision, recall, specificity, and f1_score
    p = precision_score(test_y_true, test_y_pred, average="macro")
    r = recall_score(test_y_true, test_y_pred, average="macro")
    f1 = f1_score(test_y_true, test_y_pred, average="macro")
#     sen,spe,_ = sss(test_y_true, test_y_pred, average="macro")
    print(test_y_true, test_y_pred)
    print("Test accuracy:", acc)
    print("Test confusion matrix: \n", CM(test_y_true, test_y_pred))
    print("Precision: ", p)
    print("Recall: ", r)
#     print("Specificity: ", spe)
    print("f1_score: ", f1)

In [7]:
# import packages
import logging
import numpy as np
import tensorflow as tf

In [8]:
"""  Load in the input - original data """

Xtr = np.load("preprocess/input/random_split/train_data.npy", allow_pickle = True)
ytr = np.load("preprocess/input/random_split/train_label.npy", allow_pickle = True)
print(Xtr.shape)
print(ytr.shape)

Xts = np.load("preprocess/input/random_split/test_data.npy", allow_pickle = True)
yts = np.load("preprocess/input/random_split/test_label.npy", allow_pickle = True)
print(Xts.shape)
print(yts.shape)

Xval = np.load("preprocess/input/random_split/val_data.npy", allow_pickle = True)
yval = np.load("preprocess/input/random_split/val_label.npy", allow_pickle = True)
print(Xval.shape)
print(yval.shape)

print("Train:")
showpercentage(np.unique(ytr, return_counts=True))
print()
print("Validation:")
showpercentage(np.unique(yval, return_counts=True))
print()
print("Test")
showpercentage(np.unique(yts, return_counts=True))

#%% 

(356, 64, 64, 64)
(356,)
(86, 64, 64, 64)
(86,)
(90, 64, 64, 64)
(90,)
Train:
0.2893258426966292 percent of the data has AD label
0.41853932584269665 percent of the data has CN label
0.29213483146067415 percent of the data has MCI label

Validation:
0.28888888888888886 percent of the data has AD label
0.4222222222222222 percent of the data has CN label
0.28888888888888886 percent of the data has MCI label

Test
0.29069767441860467 percent of the data has AD label
0.4186046511627907 percent of the data has CN label
0.29069767441860467 percent of the data has MCI label


In [9]:
""" Dimenstion reduction"""

# reshape the input
X_train = Xtr.reshape(-1,64,64,64,1) 
X_test = Xts.reshape(-1,64,64,64,1) 
X_val = Xval.reshape(-1,64,64,64,1) 

# traninng convolutional autoencoder(CAE)
cae = run_cae(X_train, X_val)

# use CAE to predict the train, validation and test data
encoder = Model(cae.input, cae.get_layer('bottleneck').output)

# predict
Xtr_dr = encoder.predict(X_train)
Xval_dr = encoder.predict(X_val)
Xts_dr = encoder.predict(X_test)

# check the new input shape
print(Xtr_dr.shape)
print(Xval_dr.shape)
print(Xts_dr.shape)


Instructions for updating:
Use tf.cast instead.
Train on 356 samples, validate on 90 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
356/356 [==============================] - 8s 23ms/sample - loss: 0.1626 - val_loss: 0.1546
Epoch 2/100
356/356 [==============================] - 4s 11ms/sample - loss: 0.1122 - val_loss: 0.1247
Epoch 3/100
356/356 [==============================] - 4s 11ms/sample - loss: 0.0743 - val_loss: 0.0903
Epoch 4/100
356/356 [==============================] - 4s 11ms/sample - loss: 0.0607 - val_loss: 0.0714
Epoch 5/100
356/356 [==============================] - 4s 11ms/sample - loss: 0.0538 - val_loss: 0.0612
Epoch 6/100
356/356 [==============================] - 4s 11ms/sample - loss: 0.0482 - val_loss: 0.0553
Epoch 7/100
356/356 [==============================] - 4s 11ms/sample - loss: 0.0419 - val_loss: 0.0493
Epoch 8/100
356/356 [==============================] - 4s 11ms/sample - loss: 0.0355 - val_loss: 0.0422
Epoch 9/100
356/356 [======

In [10]:
"""  NC vs. AD - dimension reduction """

# create input for binary classification of NC vs. AD
Xtr_ncad, ytr_ncad = data_filter(Xtr_dr, ytr, 1)
Xval_ncad, yval_ncad = data_filter(Xval_dr, yval, 1)
Xts_ncad, yts_ncad = data_filter(Xts_dr, yts, 1)

# rename the input
X_train = Xtr_ncad
X_test = Xts_ncad
X_val = Xval_ncad

# one hot encode the target labels 
y_train = onehot_bi(ytr_ncad)
y_test = onehot_bi(yts_ncad)
y_val = onehot_bi(yval_ncad)


# model training
model, hist = run_adpbase(X_train, y_train, X_val, y_val, lr = 0.001)

(array([0, 2]), array([149, 103]))
(array([0, 2]), array([38, 26]))
(array([0, 2]), array([36, 25]))
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train on 252 samples, validate on 64 samples
Epoch 1/100
252/252 [==============================] - 2s 7ms/sample - loss: 0.8057 - acc: 0.5992 - val_loss: 5.5651 - val_acc: 0.5938
Epoch 2/100
252/252 [==============================] - 0s 357us/sample - loss: 0.6368 - acc: 0.6905 - val_loss: 5.0209 - val_acc: 0.5938
Epoch 3/100
252/252 [==============================] - 0s 345us/sample - loss: 0.6473 - acc: 0.6865 - val_loss: 4.9133 - val_acc: 0.5938
Epoch 4/100
252/252 [==============================] - 0s 348us/sample - loss: 0.6373 - acc: 0.6746 - val_loss: 3.9582 - val_acc: 0.5938
Epoch 5/100
252/252 [==============================] - 0s 349us/sample - loss: 0.5627 - acc: 0.7222 - val_loss: 2.4922 - val_acc: 0.5938
Epoch 6/100
252/252 [==============================] - 0

In [2]:
!pip install pydot
!pip install pydotplus
!pip install graphviz

You are using pip version 9.0.1, however version 22.0.4 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 286kB 24.7MB/s ta 0:00:01
  Running setup.py bdist_wheel for pydotplus ... done
  Stored in directory: /home/ma-user/.cache/pip/wheels/ad/d2/dc/4b6affd6cb0f0c43f30fb56830bcbb807b2b3a484990a9f0d5
Successfully built pydotplus
You are using pip version 9.0.1, however version 22.0.4 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 22.0.4 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
from keras.layers import Input, Convolution2D, Flatten, Dense, Activation
from keras.models import Sequential
from keras.optimizers import SGD , Adam
# from keras.initializations import normal
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

import keras
import pydot as pyd
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

keras.utils.vis_utils.pydot = pyd


# apply a 3x3 convolution with 64 output filters on a 256x256 image:
model = Sequential()
model.add(Convolution2D(64, 3, 3, border_mode='same', dim_ordering='th',input_shape=(3, 256, 256)))
# now model.output_shape == (None, 64, 256, 256)

# add a 3x3 convolution on top, with 32 output filters:
model.add(Convolution2D(32, 3, 3, border_mode='same', dim_ordering='th'))
# now model.output_shape == (None, 32, 256, 256)
adam = Adam(lr=1e-6)
model.compile(loss='mse',optimizer=adam)
print("We finish building the model")

plot_model(model, to_file='test1.png', show_shapes=True)

We finish building the model


/home/ma-user/anaconda3/envs/TensorFlow-1.13.1/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), input_shape=(3, 256, 2..., padding="same", data_format="channels_first")`
/home/ma-user/anaconda3/envs/TensorFlow-1.13.1/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", data_format="channels_first")`
